In [ ]:
from google.colab import drive, userdata
import os
import sys

drive.mount('/content/drive')
userdata.get('HF_TOKEN')

Mounted at /content/drive


In [ ]:
!pip install transformers>=4.21.0 --quiet
!pip install torch torchvision torchaudio --quiet
!pip install tqdm --quiet
!pip install datasets --quiet
!pip install datatrove --quiet

try:
    !pip install flash-attn --no-build-isolation --quiet
    print("Flash Attention がインストールされました")
except:
    print("Flash Attention のインストールに失敗 - 標準Attentionを使用します")

依存関係をインストール中...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Flash Attention がインストールされました


In [ ]:
#Google Drive内のモジュールを利用するために作業ディレクトリを変更し、pathに追加する
lightlm_path = '/content/drive/MyDrive/LightLM'

os.chdir(lightlm_path)
print(f"作業ディレクトリ: {os.getcwd()}")

if lightlm_path not in sys.path:
    sys.path.insert(0, lightlm_path)

作業ディレクトリ: /content/drive/MyDrive/LightLM-main


In [ ]:
# デフォルトのチェックポイントパス（train.pyから）
default_checkpoint = lightlm_path + "/model_testing/model.checkpoint.epoch0_step23500_global23500.pt"
model_dir = lightlm_path + "/hf_model"
repo_name = "user/repo"
private = False

In [ ]:
#モデルのコンバートを実施する
import os
import torch
import json
import shutil
from transformers import AutoTokenizer
from model import Transformer, ModelConfig
from convert_to_hf import convert_checkpoint_to_hf

if not os.path.exists(default_checkpoint):
    print(f"❌ Checkpoint not found: {default_checkpoint}")
    print("Available checkpoints:")
    checkpoint_dir = lightlm_path + "/model_testing"
    if os.path.exists(checkpoint_dir):
        for file in os.listdir(checkpoint_dir):
            if file.endswith('.pt'):
                print(f"   - {os.path.join(checkpoint_dir, file)}")


print(f"🚀 Converting checkpoint to HuggingFace format...")
output_dir = convert_checkpoint_to_hf(default_checkpoint)
print(f"🎉 Done! Model is ready for upload to HuggingFace Hub.")
print(f"    Next step: Run 'python upload_to_hub.py' to upload to HuggingFace")

In [ ]:
import os
import json
from transformers import AutoTokenizer
from huggingface_hub import HfApi
from upload_to_hub import upload_to_huggingface

token = os.getenv('HF_TOKEN')

try:
    url = upload_to_huggingface(
        model_dir=model_dir,
        repo_name=repo_name,
        private=private,
        token=token
    )
    print(f"🎉 Success! Your model is now available at: {url}")

except Exception as e:
    print(f"❌ Upload failed: {e}")
    print("   Make sure you have:")
    print("   1. Valid HuggingFace token")
    print("   2. Proper permissions")
    print("   3. Unique repository name")